In [2]:
import nltk
from nltk.corpus import brown

nltk.download('brown')

# Load the tagged sentences from the 'news' category
news_sentences = brown.tagged_sents(categories='news')

# Calculate the split index for the last 10% of the sentences
split_index = int(len(news_sentences) * 0.9)

# Divide the sentences into training and test sets
train_set = news_sentences[:split_index]
test_set = news_sentences[split_index:]

# Optionally, you can print the sizes of the train and test sets
print(f"Training set size: {len(train_set)} sentences")
print(f"Test set size: {len(test_set)} sentences")

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\inbar\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


Training set size: 4160 sentences
Test set size: 463 sentences


In [51]:
import pandas as pd
import ex3

In [33]:
list_of_pd = []
for i in range(len(train_set)):
    list_of_pd.append(pd.DataFrame(train_set[i]))

In [34]:
list_of_pd[1]

,0,1
0,The,AT
1,jury,NN
2,further,RBR
3,said,VBD
4,in,IN
5,term-end,NN
6,presentments,NNS
7,that,CS
8,the,AT
9,City,NN-TL


In [35]:
df_train = pd.concat(list_of_pd)

In [36]:
df_train.columns = ["name","tag"]

In [37]:
count_per_tag = df_train.groupby(["name", "tag"]).size().reset_index(name='appearances')


In [38]:
count_per_word = df_train.groupby(["name"]).size().reset_index(name='total_word_appearances')


In [39]:
count_per_tag = count_per_tag.merge(count_per_word,on ="name",how = "left")

In [40]:
count_per_tag["rate"] = count_per_tag["appearances"] /count_per_tag["total_word_appearances"]

In [41]:
count_dict = {}

for _, row in count_per_tag.iterrows():
    word = row['name']
    tag = row['tag']
    appearances = row['rate']
    
    if word not in count_dict:
        count_dict[word] = []
    
    # Append the (tag, appearances) tuple
    count_dict[word].append((tag, appearances))

# Sort each list of tags by appearances in descending order
for word in count_dict:
    count_dict[word] = sorted(count_dict[word], key=lambda x: x[1], reverse=True)


In [42]:
list_of_pd_test = []
for i in range(len(test_set)):
    list_of_pd_test.append(pd.DataFrame(test_set[i]))
df_test = pd.concat(list_of_pd_test)

In [43]:
df_test.columns = ["name","tag"]

In [44]:
def get_prob(word):
    if count_dict.get(word):
        return count_dict[word][0][0]
    else:
        return "NN"

In [45]:
df_test["predicted tag"] = df_test.apply(lambda x:get_prob(x["name"]),axis = 1)

In [46]:
accuracy = len(df_test[df_test["tag"] == df_test["predicted tag"]]) / len(df_test)

In [49]:
accuracy

0.8377354729393003

In [47]:
error_rate = 1 - accuracy

In [48]:
error_rate

0.16226452706069971

In [55]:
train_set[0]

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]